<a href="https://colab.research.google.com/github/anselmo-pitombeira/Notebooks/blob/master/Modelo_Calcados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problema de mix de calçados

Neste notebook, descrevemos como resolver um problema de decisão de mix de produção de calçados. O problema é formulado como um modelo de programação linear, e em seguida é resolvido com uso da biblioteca Pyomo para Python e o solver COIN-OR CBC.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/anselmo-pitombeira/Notebooks/blob/master/Modelo_Calcados.ipynb)

In [0]:
!pip install pyomo


     |████████████████████████████████| 2.4MB 4.9MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 256kB 40.1MB/s 
     |████████████████████████████████| 163kB 54.8MB/s 


In [0]:
import pyomo.environ as pyo
import numpy as np

##Dados do problema
lucro = np.array([300,400,200,150])
A = np.array([[0.30,0.30,0.25,0.15],
              [0.25,0.35,0.30,0.10],
              [0.45,0.50,0.40,0.22],
              [0.15,0.15,0.10,0.05]])

horas = np.array([1000,1000,1000,1000])
demanda = np.array([800,750,600,500])




In [0]:
modelo = pyo.ConcreteModel()

##Variáveis de decisão
n = 4    ##Número de variáveis

modelo.x = pyo.Var([i for i in range(n)], domain=pyo.NonNegativeReals)

##Função objetivo
expr = sum(lucro[i]*modelo.x[i] for i in range(n))
modelo.obj = pyo.Objective(expr = expr, sense=pyo.maximize)

##Restrições horas
modelo.r1 = pyo.Constraint(expr=sum(A[0,i]*modelo.x[i] for i in range(n))<=horas[0])
modelo.r2 = pyo.Constraint(expr=sum(A[1,i]*modelo.x[i] for i in range(n))<=horas[1])
modelo.r3 = pyo.Constraint(expr=sum(A[2,i]*modelo.x[i] for i in range(n))<=horas[2])
modelo.r4 = pyo.Constraint(expr=sum(A[3,i]*modelo.x[i] for i in range(n))<=horas[3])

##Restrições demanda
modelo.d1 = pyo.Constraint(expr=modelo.x[0] <= demanda[0])
modelo.d2 = pyo.Constraint(expr=modelo.x[1] <= demanda[1])
modelo.d3 = pyo.Constraint(expr=modelo.x[2] <= demanda[2])
modelo.d4 = pyo.Constraint(expr=modelo.x[3] <= demanda[3])

modelo.pprint()

1 Set Declarations
    x_index : Dim=0, Dimen=1, Size=4, Domain=None, Ordered=False, Bounds=(0, 3)
        [0, 1, 2, 3]

1 Var Declarations
    x : Size=4, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 300*x[0] + 400*x[1] + 200*x[2] + 150*x[3]

8 Constraint Declarations
    d1 : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf : x[0] : 800.0 :   True
    d2 : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf : x[1] : 750.0 :   True
    d

In [0]:
!apt-get install coinor-cbc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5
  coinor-libosi1v5
The following NEW packages will be installed:
  coinor-cbc coinor-libcbc3 coinor-libcgl1 coinor-libclp1
  coinor-libcoinutils3v5 coinor-libosi1v5
0 upgraded, 6 newly installed, 0 to remove and 25 not upgraded.
Need to get 2,737 kB of archives.
After this operation, 8,130 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 coinor-libcoinutils3v5 amd64 2.10.14+repack1-1 [472 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 coinor-libosi1v5 amd64 0.107.9+repack1-1 [270 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 coinor-libclp1 amd64 1.16.11+repack1-1 [863 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 coinor-libcgl1 amd64 0.59.10+repack1-1 [433 kB]
Ge

In [0]:
solver = pyo.SolverFactory("cbc", executable="/usr/bin/cbc")
results = solver.solve(modelo)
##results.write()

In [0]:
print("Valor ótimo lucro = ", modelo.obj.value())
print("Solução ótima = ", modelo.x.get_values())

    deprecated. Use the .expr property getter instead
Valor ótimo lucro =  692500.0
Solução ótima =  {0: 800.0, 1: 750.0, 2: 387.5, 3: 500.0}
